<a href="https://colab.research.google.com/github/emadens/Object-Detection/blob/main/EfficientDet/EfficientDet_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientDet model implementation with custom dataset.

### Import Drive into working directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')